# OpenAI Parameters

# Overview
When making requests to OpenAI models, several parameters can be used to control the behavior and output of the model. \
Understanding these parameters helps in fine-tuning the responses to meet specific requirements, whether for generating text, answering questions, or any other use case.

For more detailed examples, refer to the official documentation [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [3]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True)

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"
SEED = 123

# Parameter: max_tokens
**Description**: The maximum number of tokens to generate in the completion. \
**Default**: 16 \
**Example**: max_tokens=50

The token count of your prompt plus max_tokens can't exceed the model's context length. \
Most models have a context length of 2048 tokens (except for the newest models, which support 4096). Please refer to documentation.

In [4]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=max_tokens
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [16, 32, 60, 100]
for penalty in penalties:
    print(f"Max Tokens: {penalty}\n")
    print(call_openai_with_max_tokens(penalty))
    print("\n" + "-"*80 + "\n")

Max Tokens: 16

The best pet often depends on an individual's preferences, lifestyle, and living situation.

--------------------------------------------------------------------------------

Max Tokens: 32

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Some people might consider dogs the best pets due to their loyalty and companionship,

--------------------------------------------------------------------------------

Max Tokens: 60

The best pet can vary greatly depending on individual preferences and lifestyles! Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often called "man's best friend." They require regular exercise and social interaction.

2. **Cats**: Independent but affectionate,

--------------------------------------------------------------------------------

Max Tokens: 100

The best pet depends on individual preferences, lifestyles, and living situations. Here are a 

# Parameter: temperature

**Description**: Controls the randomness of the output. Lower values make the output more deterministic, while higher values increase randomness. \
**Value Range**: 0 to 1 \
**Default Value**: 1 \
**Example**: temperature=0.7

Higher values means the model will take more risks. \
Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.

---
**NOTE**: We generally recommend altering this or top_p but not both.


In [13]:
def call_openai(num_times, prompt, temperature=0.75, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model= CHAT_COMPLETIONS_MODEL,
                messages = [
                    {"role": "system", "content": "You're a helpful assistant that don't uses lists to response"},
                    {"role": "user", "content": prompt},
                ],
                max_tokens=60,
                seed=SEED,
                temperature = temperature
            )
        else:
            response = client.chat.completions.create(
                model= CHAT_COMPLETIONS_MODEL,
                messages = [
                    {"role": "system", "content": "You're a helpful assistant that don't uses lists to response"},
                    {"role": "user", "content": prompt},
                ],
                max_tokens=60,
                temperature = temperature
            )
        print(response.choices[0].message.content)

In [14]:
# Without seed and temperature, the response is different each time
call_openai(10, 'The best pet is a ')

The best pet often depends on personal preferences and lifestyle. Some people may find dogs to be the best due to their loyalty and companionship, while others might prefer cats for their independence and low maintenance. Other pets, like birds, rabbits, or fish, can also be wonderful companions, each bringing their unique
The best pet can vary greatly depending on individual preferences and lifestyles. Some people might find that dogs are the best due to their loyalty and companionship, while others may prefer cats for their independence and affection. Small pets like hamsters or guinea pigs can be great for those with limited space, while fish can
The best pet often depends on individual preferences and lifestyles. Some people may find that dogs are the best companions due to their loyalty and energy, while others might prefer cats for their independence and low maintenance. Some enjoy the company of smaller pets like rabbits or hamsters, which can be great for those with
The best pe

In [15]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, 'The best pet is a ', temperature = 0, use_seed=True)

The best pet often depends on individual preferences and lifestyles. Some people might find that dogs are the best due to their loyalty and companionship, while others may prefer cats for their independence and low maintenance. Small animals like hamsters or rabbits can be great for those with limited space, and fish can provide a
The best pet often depends on individual preferences and lifestyles. Some people might find that dogs are the best due to their loyalty and companionship, while others may prefer cats for their independence and low maintenance. Small animals like hamsters or rabbits can be great for those with limited space, while fish can provide a
The best pet often depends on individual preferences and lifestyles. Some people might find that dogs are the best due to their loyalty and companionship, while others may prefer cats for their independence and low maintenance. Small animals like hamsters or rabbits can be great for those with limited space, while fish can provide

# Parameter: n
**Description**: Specifies the number of completions to generate for each prompt. \
**Default Value**: 1 \
**Example**: n = 3 

---
**Note**: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

In [19]:
response = client.chat.completions.create(
    model= CHAT_COMPLETIONS_MODEL,
    messages = [
        {"role": "system", "content": "You're a helpful assistant that don't uses lists to response"},
        {"role": "user", "content": "The best pet is a "},
    ],
    max_tokens=60,
    seed=SEED,
    temperature = 0.9,
    n = 3
)

for index, c in enumerate(response.choices):
    print(index + 1, c.message.content)

1 The best pet is often considered to be one that suits your lifestyle, preferences, and living situation. For some, a dog offers companionship and loyalty, while for others, a cat provides a more independent yet affectionate presence. Small pets like hamsters or rabbits can be delightful for those with limited space.
2 The best pet often depends on individual preferences and lifestyles. Some people might find that dogs, with their loyalty and companionship, make the best pets for their active and social nature. Others may prefer cats, which can offer a sense of independence and are typically lower maintenance. There are also those who enjoy the
3 The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Some people find dogs to be the best companions due to their loyalty and ability to bond with humans. Others might prefer cats for their independence and low-maintenance nature. Exotic pets, such as reptiles or birds, can


# Parameter: presence_penalty
**Description**: Penalizes new tokens based on whether they appear in the text so far, encouraging the model to use new tokens. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: presence_penalty=0.5

In [21]:
def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant that dont uses lists to response."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=60,
                    presence_penalty= presence_penalty
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")

Presence Penalty: 0

The best pet often depends on individual preferences and lifestyle. Some people might favor dogs for their loyalty and companionship, while others might prefer cats for their independence and lower maintenance needs. Additionally, small animals like hamsters or rabbits might appeal to those looking for a pet that requires less space. Ultimately, the

--------------------------------------------------------------------------------

Presence Penalty: 0.5

The best pet often depends on an individual's lifestyle, preferences, and living situation. Some people may find dogs to be the best companions due to their loyalty and ability to bond closely with humans. Others might prefer cats for their independence and low-maintenance care. Some individuals enjoy the uniqueness of exotic pets such

--------------------------------------------------------------------------------

Presence Penalty: 1.0

The best pet really depends on an individual's lifestyle, preferences, and li

# Parameter: frequency_penalty
**Description**: Penalizes new tokens based on their existing frequency in the text so far, reducing the likelihood of repeating the same line verbatim. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: frequency_penalty=0.5

#### Use cases to explore
1. **Compare Responses** \
Generate multiple completions to compare and choose the best response for your use case.

2. **Increase Diversity** \
Use multiple completions to get a variety of responses, which is useful in creative applications.

3. **Enhance Robustness** \
Generate multiple responses to ensure consistency and accuracy across different completions.

#### Best Practices
1. **Optimize Prompt Length** \
Keep your prompts concise but informative to ensure the model has enough context.

2. **Adjust Temperature and Top_p** \
Use these parameters to balance between deterministic and creative responses.

3. **Monitor Token Usage** \
Be mindful of the max_tokens parameter to manage costs and response length.

4. **Use Stopping Sequences** \
Define stopping sequences to control where the model should stop generating text, ensuring the output is within the desired context.

5. **Generate Multiple Completions** \
Use the n parameter to generate multiple completions and select the best one for your needs.

## Exercise 1: Exploring 'top_p' Parameter
### Description:
- The 'top_p' parameter controls nucleus sampling. Lower values make the output more focused by limiting the pool of tokens.

### Task:
- Write a function that calls the model with different 'top_p' values and observe how the output changes.


In [23]:
def call_openai_with_top_p(top_p):
    response = client.chat.completions.create(
        model= CHAT_COMPLETIONS_MODEL,
        messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": "Describe the benefits of exercise."}
        ],
        max_tokens=60,
        top_p= top_p      
    )
    return response.choices[0].message.content

# Try with different values of top_p
top_p_values = [0.1, 0.3, 0.7, 1.0]
for value in top_p_values:
    print(f"Top P: {value}\n")
    print(call_openai_with_top_p(value))
    print("\n" + "-"*80 + "\n")


Top P: 0.1

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits:
1. **Improved Cardiovascular Health**: Regular exercise strengthens the heart, improves circulation, and can lower blood pressure and cholesterol levels, reducing the risk of heart

--------------------------------------------------------------------------------

Top P: 0.3

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits:
1. **Improved Cardiovascular Health**: Regular exercise strengthens the heart, improves circulation, and helps lower blood pressure and cholesterol levels.
2. **Weight Management

--------------------------------------------------------------------------------

Top P: 0.7

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits
1

## Exercise 2: Frequency Penalty Exploration
### Description:
- This exercise will explore how the 'frequency_penalty' parameter affects the model's tendency to repeat itself.

### Task:
- Generate completions using different frequency_penalty values and note the diversity in responses.



In [25]:
def call_openai_with_frequency_penalty(frequency_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Tell me a fun fact about space."}],
                    max_tokens=60,
                    frequency_penalty= frequency_penalty
    )
    return response.choices[0].message.content 

penalties = [-1.0, 0.0, 1.0, 2.0]
for penalty in penalties:
    print(f"Frequency Penalty: {penalty}\n")
    print(call_openai_with_frequency_penalty(penalty))
    print("\n" + "-"*80 + "\n")


Frequency Penalty: -1.0

Sure! One fun fact about space is that it is completely silent. Unlike on Earth, where sound travels through air, space is a near-perfect vacuum, which means there are very few molecules to carry sound waves. As a result, if you were to be in space, you wouldn't hear anything,

--------------------------------------------------------------------------------

Frequency Penalty: 0.0

A fun fact about space is that it is completely silent! Unlike on Earth, where sound travels through air, space is a vacuum, meaning there are no molecules to carry sound waves. As a result, if you were in space and someone shouted, you wouldn’t be able to hear them at all

--------------------------------------------------------------------------------

Frequency Penalty: 1.0

A fun fact about space is that it has a distinct smell! Astronauts who have returned from spacewalks have reported that their suits carry a peculiar odor, often described as a mix of hot metal, seared steak, a

## Exercise 3: Multi-Completion with 'n' Parameter
### Description:
- This exercise will help students generate multiple completions for the same prompt and compare the results.

### Task:
- Generate 3 completions for the same prompt by adjusting the 'n' parameter.


In [27]:
response = client.chat.completions.create(
    model= CHAT_COMPLETIONS_MODEL,
    messages=[
        {"role":"system", "content":"You are a helpful assistant."},
        {"role":"user","content": "Tell me a fun fact about cheese"}
        ],
        n=3,
        temperature=0.5,
        max_tokens=100,
)

for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 A fun fact about cheese is that there are over 1,800 different types of cheese produced around the world! Each variety has its own unique flavor, texture, and aroma, influenced by factors such as the type of milk used (cow, goat, sheep), the region where it's made, and the production methods. For example, Italy is famous for its Parmigiano-Reggiano, while France is known for its Brie and Roquefort. This diversity makes cheese a fascinating subject for
1 Did you know that cheese is one of the oldest man-made foods? Archaeological evidence suggests that cheese-making dates back over 7,200 years to around 5500 BC! The earliest evidence of cheese was found in ancient pottery in Poland, indicating that our ancestors were already enjoying this delicious dairy product thousands of years ago. Plus, there are over 1,800 different types of cheese produced worldwide, each with its own unique flavor and texture!
2 A fun fact about cheese is that there are over 2,000 different types of cheese pro

## Exercise 4: Temperature vs. Deterministic Output
### Description:
- Adjust the 'temperature' parameter to explore the balance between creative and deterministic responses.

### Task:
- Generate completions with different temperature values and analyze the variance in output.


In [30]:
def call_openai_with_temperature(temperature):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Suggest 3 names for a new tech startup."}],
                    max_tokens=100,
                    temperature= temperature
    )
    return response.choices[0].message.content

temperatures = [0.0, 0.3, 0.7, 1.0]
for temp in temperatures:
    print(f"Temperature: {temp}\n")
    print(call_openai_with_temperature(temp))
    print("\n" + "-"*80 + "\n")

Temperature: 0.0

Sure! Here are three name suggestions for your new tech startup:

1. **Innovatech Solutions**
2. **NexGen Dynamics**
3. **Quantum Leap Technologies**

Feel free to mix and match or modify these names to better fit your vision!

--------------------------------------------------------------------------------

Temperature: 0.3

Sure! Here are three name suggestions for your tech startup:

1. **Innovatech Solutions**
2. **NexGen Dynamics**
3. **Quantum Leap Technologies**

Feel free to mix and match or modify these names to better fit your vision!

--------------------------------------------------------------------------------

Temperature: 0.7

Sure! Here are three suggestions for a tech startup name:

1. **Innovatech Solutions**
2. **NexSphere Technologies**
3. **QuantumLeap Innovations**

Feel free to modify these or ask for more options!

--------------------------------------------------------------------------------

Temperature: 1.0

Sure! Here are three suggesti